<a href="https://colab.research.google.com/github/parvpareek/hindi-text-summarization/blob/main/alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip installations
!pip install huggingface-cli datasets transformers
!pip install matplotlib seaborn sklearn
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This

In [2]:
from datasets import load_dataset
dataset = load_dataset("parvpareek/pitt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/439 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/55 [00:00<?, ? examples/s]

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Sample text
text = "Replace this text with your own text."

# Tokenize text
tokens = tokenizer(text, return_tensors="pt")

# Pass input through BERT model
with torch.no_grad():
    outputs = model(**tokens)

# Extract embeddings from BERT model outputs
#embeddings = outputs.last_hidden_state

# Print shape of embeddings
#print("Shape of embeddings:", embeddings.shape)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

NameError: name 'torch' is not defined

In [5]:
def tokenize_function(example):

  tokenized_input = tokenizer(example["transcript"], padding=True, truncation=True, max_length=512, return_tensors='pt')
  example["input_ids"] = tokenized_input.input_ids
  example["attention_mask"] = tokenized_input.attention_mask
  #example["length"] = [sum(x) for x in tokenized_input.attention_mask]
  #example["tl"] = [len(x) for x in example["transcript"]]
  return example


In [6]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=8, remove_columns=["patient_id", "transcript"])

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset = tokenized_dataset.rename_column("category", "label")

ValueError: Original column name category not in the dataset. Current columns in the dataset: ['label', 'input_ids', 'attention_mask']

In [12]:
from datasets import DatasetDict, Dataset

split_size = 0.1
sub_train_size = int(tokenized_dataset["train"].num_rows * split_size)
sub_test_size = int(tokenized_dataset["test"].num_rows * split_size)
sub_val_size = int(tokenized_dataset["validation"].num_rows * split_size)

train_subset = Dataset.from_dict(tokenized_dataset["train"].shuffle(seed=42)[:sub_train_size])
test_subset = Dataset.from_dict(tokenized_dataset["test"].shuffle(seed=42)[:sub_test_size])
val_subset = Dataset.from_dict(tokenized_dataset["validation"].shuffle(seed=42)[:sub_val_size])


In [13]:
subset = DatasetDict({
    'train': train_subset,
    'test': test_subset,
    'validation': val_subset
})
subset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 43
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
})

# Training

In [76]:
from transformers import TrainingArguments

model_name = 'bert-base-cased'
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,
    eval_strategy="epoch",# number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [77]:
import evaluate

metric = evaluate.load("accuracy")


In [78]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [80]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=subset["train"],
    eval_dataset=subset["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(torch.optim.AdamW(model.parameters(), lr=2e-5), None),

)

In [81]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.773405,0.400000
2,0.639300,0.640287,0.600000
3,0.639300,0.607140,0.800000
4,0.477000,0.586579,0.800000
5,0.296500,0.530963,0.800000
6,0.296500,0.542927,0.800000
7,0.209100,0.512207,0.800000
8,0.209100,0.512869,0.800000
9,0.164000,0.515053,0.800000
10,0.143500,0.521865,0.800000


TrainOutput(global_step=60, training_loss=0.32156895399093627, metrics={'train_runtime': 1858.0437, 'train_samples_per_second': 0.231, 'train_steps_per_second': 0.032, 'total_flos': 81826510439400.0, 'train_loss': 0.32156895399093627, 'epoch': 10.0})

In [83]:
trainer.evaluate()

{'eval_loss': 0.5218647122383118,
 'eval_accuracy': 0.8,
 'eval_runtime': 3.1612,
 'eval_samples_per_second': 1.582,
 'eval_steps_per_second': 0.316,
 'epoch': 10.0}

# Custom Training Loop

In [10]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm


In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [17]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    subset["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    subset["validation"], collate_fn=data_collator, batch_size=batch_size
)


In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)


In [19]:
from transformers import get_scheduler

num_train_epochs = 20
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    eval_loss = 0
    eval_steps = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch in eval_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss
            eval_loss += loss.item()
            eval_steps += 1

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct_predictions += (predictions == inputs["labels"]).sum().item()

    avg_eval_loss = eval_loss / eval_steps
    accuracy = correct_predictions / len(subset["validation"])
    print(f"Epoch: {epoch + 1}, Eval Loss: {avg_eval_loss}, Accuracy: {accuracy}")


  0%|          | 0/120 [00:00<?, ?it/s]

Epoch: 1, Eval Loss: 0.714084804058075, Accuracy: 0.4
Epoch: 2, Eval Loss: 0.7163178324699402, Accuracy: 0.2
Epoch: 3, Eval Loss: 0.7075097560882568, Accuracy: 0.4


In [ ]:
tokenizer_vocab_size = len(tokenizer.get_vocab())
tokenizer_vocab_size

In [ ]:
embedding_weight_size = model.get_input_embeddings().weight.size(0)

In [ ]:
if tokenizer_vocab_size > embedding_weight_size:
    raise ValueError("The vocabulary size of the tokenizer is larger than the size of the embedding weight matrix in the model.")

In [ ]:
embedding_weight_size

In [ ]:
new_embedding_weight = torch.zeros(tokenizer_vocab_size, embedding_weight_size)
new_embedding_weight[:embedding_weight_size] = model.get_input_embeddings().weight
model.get_input_embeddings().weight = torch.nn.Parameter(new_embedding_weight)

In [29]:
[x < tokenizer.vocab_size for x in tokenized_dataset["train"]["input_ids"]]

TypeError: '<' not supported between instances of 'list' and 'int'

In [38]:
t = [torch.tensor(x) for x in tokenized_dataset["train"]["input_ids"]]

RuntimeError: stack expects each tensor to be equal size, but got [182] at entry 0 and [145] at entry 8

In [41]:
for i,x in enumerate(t):
  print(i)
  assert torch.all(x < tokenizer.vocab_size )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


AssertionError: 

In [68]:
for i, x in enumerate(dataset["validation"]["transcript"]):
  to = tokenizer(x, return_tensors="pt")
  z = torch.tensor(to["input_ids"])

  if torch.all(z < tokenizer.vocab_size):
    print(i)

  else:
    print(z)
    print(tokenizer.decode(z))
    print(x)

<ipython-input-68-793c4f93118f>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z = torch.tensor(to["input_ids"])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


439


In [75]:
print(tokenizer.decode(tokenized_dataset["train"]["input_ids"][0]))

[CLS] an average home that looks very much like ours at many times gram and the mother is well she's spilling her water which is not very good but she's doing washing dishes and drying them and one of the children upset the cookie jar and that's an a f p very average thing to go in on s r in a home our home is a very it's a very active family and so that it makes it kinda hard at our house sometime because uh there's a lot of and ac like i said a lot of activity well the poor girl is spilling the water on her feet it's gonna be and k wet but then she'll hafta struggle with that also there are a few more dishes left that didn't get finished this hasta be the dishes and how that little boy fell off that stool i don't know but he did no [SEP] [PAD] [PAD] [PAD]
